In [1]:
#======================================================================================================
# by kobongsoo, 
# ============= HISTORY =============================================
# make by: 2022-04-13
# updata: 2022-09-27
# => KlueNLI 데이터 셋 추가
# => 데이터 셋은 KorNLI(550,152) 가 KlueNLI(24,998)보다 20배정도 크지만, 
# Klue 데이터 셋이 실제 모델 훈련에는 더 좋다고 함.
# ======================================================================
#
# sentence-bert NLI 훈련 및 평가 예시
# => 기존 (distil)bert 모델을 가지고, NLI로 훈련 및 평가 후, S-BERT로 만드는 예시임.
#
#=> 필요에 따라 출력 dimension을 768보다 작게 줄이고 싶을때 dense 모델을 추가해서 줄일수 있음
#=> reduce_out_dimension = True 로 하면, 출력 임베딩 dimension이 줄어들게 설정가능함
#
# => sentence-transformers 패키지를 이용하여 구현 함.(*pip install -U sentence-transformers 설치 필요)
#
# 도큐먼트 : https://www.sbert.net/index.html
# 소스참고 : https://github.com/BM-K/KoSentenceBERT-ETRI

# pip install -U sentence-transformers
#======================================================================================================
import torch.nn as nn
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import sys
import os
import gzip
import csv

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert", logfilename="../../log/s-bert")
device = GPU_info()
seed_everything(111)

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


logfilepath:../../log/s-bert_2022-10-13.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
# s-bert로 만들 원본 bert 경로
model_path = "../../data11/model/sbert/sbert-mdistilbertV3.1.1-disitl-ns-1"

# 원본 bert를 sentencebert로 만든후 만들어진 s-bert 저장 경로
# => **해당 경로\eval 폴더에 similarity_evaluation_sts-dev_result.csv 파일로 각 epoch 마다 평가된 결과가 기록된다.
#smodel_path = 'output/training_nli_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
smodel_path = "../../data11/model/sbert/sbert-mdistilbertV3.1.1-disitl-ns-1.5"

use_kor = True   #kornli 훈련하는 경우 == True
use_klue = True  #kluenli 훈련하는 경우 == True

# KorNLI, KorSTS 파일 경로
train_kornli_file = '../../data11/korpora/kornli/snli_1.0_train.ko.tsv'
eval_korsts_file = '../../data11/korpora/korsts/tune_dev.tsv'

# KLUENIL, KlueSTS 파일 경로
train_kluenli_file = '../../data11/korpora/klue-nli/klue-nli-v1.1_train.json'
eval_kluests_file = '../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json'

train_batch_size = 256
num_epochs = 8          # 8 정도면 최상의 값 찾을수 있음
max_seq_length = 128
lr = 3e-5  # default=2e-5
eps = 1e-6
# * True=모델에 상관없이 영어는 소문자로 입력하겠다는 뜻(glue는 소문자로만 비교하는게 더 적확함), 한국어는 True도 상관없음
#do_lower_case = True 
do_lower_case = False  
#============================================================================
# *출력 dimension을 줄일 경우에는 True로 하고, out_dimension에 줄일 값을 설정함
reduce_out_dimension = False  # True이면 dimension을 줄임=>Dense 모델 추가됨
out_dimension = 128
#============================================================================

# 모델과 tokenizer 를 불러옴
# => **사전파일(vocab.txt, *.json) 와 model 경로(config.json, pytorch_model.bin)가 같은 경로에 있어야 함.
word_embedding_model = models.Transformer(model_path, max_seq_length=max_seq_length, do_lower_case=do_lower_case)

# word embedding_model 출력 
print(word_embedding_model)

Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 


In [3]:
# 2 bert 모델의 임베딩 풀링 정책을 설정(cls 이용, 워드임베딩 평균이용, 워드임베딩 max 이용)
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용
# pooling model 출력 
print(pooling_model)
print(pooling_model.get_sentence_embedding_dimension())

Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
768


In [4]:
# 3. dense 모델 추가(옵션)
#=> 필요에 따라 출력 dimension을 768보다 작게 줄이고 싶을때 dense 모델을 추가해서 줄임.
#=> https://www.sbert.net/docs/training/overview.html?highlight=dense 참조
if reduce_out_dimension:
    dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), # 입력 dimension은 앞에 pooling모델 embedding dimension으로 지정
                               out_features=out_dimension,  # 출력 dimension
                               activation_function=nn.Tanh())  # activation function은 Tahn으로 정의

In [5]:
# SBERT 모델 생성
if reduce_out_dimension:
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
else:
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [6]:
# 훈련 데이터 불러오기
# => [sentence1, sentence2], labels 식으로 만듬
label2int = {"entailment": 0, "neutral": 1, "contradiction": 2}
train_samples = []

####################################################################################################
# KorNLI 훈련 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_kor == True:
    count = 0
    logger.info(f"Read NLI train dataset:{train_kornli_file}")

    with open(train_kornli_file, "rt", encoding="utf-8") as f:
        lines = f.readlines()
        for line in lines:
            s1, s2, label = line.split('\t')
            label = label2int[label.strip()]
            if count < 5:
                print(f"{s1}, {s2}, {label}")
            
            train_samples.append(InputExample(texts=[s1, s2], label=label))
            count += 1
        
    logger.info(f'*kornli len: {count}')
####################################################################################################

####################################################################################################
# KlueNLI 훈련 데이터 셋 설정(.json 파일)
# => 아래처럼 load_dataset으로 불러와서 사용할수도 있음.
# datas = load_dataset("klue", "nli", split="train")
# for data in datas:
#        s1 = data["sentence1"]
#        s2 = data["sentence2"]
#        label = data["label"]["label"]
###################################################################################################    
# kluenli 훈련인 경우 
if use_klue == True:
    count = 0
    import json
    logger.info(f"Read NLI train dataset:{train_kluenli_file}")

    with open(train_kluenli_file, "rt", encoding="utf-8") as f:
        datas = json.load(f)
        for data in datas:
            #print(data)
            s1 = data["premise"].strip()
            s2 = data["hypothesis"].strip()
            label = label2int[data["gold_label"].strip()]
            if count < 5:
                print(f"{s1}, {s2}, {label}")

            train_samples.append(InputExample(texts=[s1, s2], label=label))
            count += 1
            
    logger.info(f'*kluenli len: {count}')
        
logger.info(f'*train_samples_len:{len(train_samples)}')

2022-10-13 12:32:57,698 - s-bert - INFO - Read NLI train dataset:../../data11/korpora/kornli/snli_1.0_train.ko.tsv


말을 탄 사람이 고장난 비행기 위로 뛰어오른다., 한 사람이 경쟁을 위해 말을 훈련시키고 있다., 1
말을 탄 사람이 고장난 비행기 위로 뛰어오른다., 한 사람이 식당에서 오믈렛을 주문하고 있다., 2
말을 탄 사람이 고장난 비행기 위로 뛰어오른다., 사람은 야외에서 말을 타고 있다., 0
카메라에 웃고 손을 흔드는 아이들, 그들은 부모님을 보고 웃고 있다, 1
카메라에 웃고 손을 흔드는 아이들, 아이들이 있다, 0


2022-10-13 12:33:00,380 - s-bert - INFO - *kornli len: 550152
2022-10-13 12:33:00,382 - s-bert - INFO - Read NLI train dataset:../../data11/korpora/klue-nli/klue-nli-v1.1_train.json
2022-10-13 12:33:00,559 - s-bert - INFO - *kluenli len: 24998
2022-10-13 12:33:00,561 - s-bert - INFO - *train_samples_len:575150


힛걸 진심 최고다 그 어떤 히어로보다 멋지다, 힛걸 진심 최고로 멋지다., 0
100분간 잘껄 그래도 소닉붐땜에 2점준다, 100분간 잤다., 2
100분간 잘껄 그래도 소닉붐땜에 2점준다, 소닉붐이 정말 멋있었다., 1
100분간 잘껄 그래도 소닉붐땜에 2점준다, 100분간 자는게 더 나았을 것 같다., 1
101빌딩 근처에 나름 즐길거리가 많습니다., 101빌딩 근처에서 즐길거리 찾기는 어렵습니다., 2


In [7]:
# 데이터 셋, 데이터 로더, 손실함수 정의
train_dataset = SentencesDataset(train_samples, model=model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

# train_loss 정의 
# => 기본은 MSELoss 인데, NLI 훈련시에는 MultilpleNegativesRankingLoss(MNR Loss)가 좋다고 함.
train_loss = losses.MultipleNegativesRankingLoss(model)
#train_loss = losses.SoftmaxLoss(model=model, 
#                                sentence_embedding_dimension=model.get_sentence_embedding_dimension(), 
#                                num_labels=len(label2int))

In [8]:
#Read STSbenchmark dataset and use it as development set
# 평가데이터 불러오기
#korsts 파일로 두 문장간 유사도를 수치로(5.0이 만점=매우 유사) 측정함.
dev_samples = []

####################################################################################################
# KorSTS 평가 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_kor == True:
    count = 0
    logger.info(f"Read STS dev dataset=>{eval_korsts_file}")
    with open(eval_korsts_file, 'rt', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            text_a, text_b, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
            
            if count < 5:
                print(f"{text_a}, {text_b}, {score}")
            
            dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
####################################################################################################  

####################################################################################################
# KlueSTS 평가 데이터 셋 설정(.json 파일)
# => 아래처럼 load_dataset으로 불러와서 사용할수도 있음.
# datas = load_dataset("klue", "sts", split="test")
# for data in datas:
#        text_a = data["sentence1"]
#        text_b = data["sentence2"]
#        score = data["labels"]["label"]
#        score = float(score) / 5.0  
####################################################################################################           
if use_klue == True:
    count = 0
    logger.info(f"Read STS dev dataset=>{eval_kluests_file}")
    with open(eval_kluests_file, "rt", encoding="utf-8") as f:
        datas = json.load(f)
        for data in datas:
            text_a = data["sentence1"]
            text_b = data["sentence2"]
            score = data["labels"]["label"]
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함

            if count < 5:
                print(f"{text_a}, {text_b}, {score}")

            dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
####################################################################################################  

    
# 2개의 bert 모델에서 구한 2개의 embedding 값들의 cosine 유사도를 구해서, 이를 실제 score와 비교해서 유사도 측정함
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, 
                                                                 batch_size=train_batch_size, 
                                                                 name='sts-dev')

2022-10-13 12:33:00,578 - s-bert - INFO - Read STS dev dataset=>../../data11/korpora/korsts/tune_dev.tsv
2022-10-13 12:33:00,615 - s-bert - INFO - Read STS dev dataset=>../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json


안전모를 가진 한 남자가 춤을 추고 있다., 안전모를 쓴 한 남자가 춤을 추고 있다., 1.0
어린아이가 말을 타고 있다., 아이가 말을 타고 있다., 0.95
한 남자가 뱀에게 쥐를 먹이고 있다., 남자가 뱀에게 쥐를 먹이고 있다., 1.0
한 여성이 기타를 연주하고 있다., 한 남자가 기타를 치고 있다., 0.48
한 여성이 플루트를 연주하고 있다., 남자가 플루트를 연주하고 있다., 0.55
무엇보다도 호스트분들이 너무 친절하셨습니다., 무엇보다도, 호스트들은 매우 친절했습니다., 0.9800000000000001
주요 관광지 모두 걸어서 이동가능합니다., 위치는 피렌체 중심가까지 걸어서 이동 가능합니다., 0.27999999999999997
학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018학년도 수능부터 도입된 영어 영역 절대평가는 올해도 유지한다., 영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년도부터 적용했던 EBS 연계 방식을 올해도 유지한다., 0.26
다만, 도로와 인접해서 거리의 소음이 들려요., 하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다., 0.74
형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요., 가족 모임 일정은 바꾸지 말도록 하십시오., 0.5


In [9]:

warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up

# evaluation_steps은 20%로 설정
evaluation_steps = int(len(train_dataset) * num_epochs / train_batch_size * 0.2)
logger.info(f"model:{model_path}, smodel:{smodel_path}")
logger.info("*batch_size: {}, epoch:{}, train_dataset:{}, Warmup-steps: {}, evaluation_step: {}".format(train_batch_size, num_epochs, len(train_dataset), warmup_steps, evaluation_steps))

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=evaluation_steps,
          warmup_steps=warmup_steps,
          optimizer_params= {'lr': lr, 'eps': eps, 'correct_bias': False},
          save_best_model=True, # **기본 = True : eval 가장 best 모델을 output_Path에 저장함
          output_path=smodel_path
          )


2022-10-13 12:33:00,636 - s-bert - INFO - model:../../data11/model/sbert/sbert-mdistilbertV3.1.1-disitl-ns-1, smodel:../../data11/model/sbert/sbert-mdistilbertV3.1.1-disitl-ns-1.5
2022-10-13 12:33:00,638 - s-bert - INFO - *batch_size: 256, epoch:8, train_dataset:575150, Warmup-steps: 1798, evaluation_step: 3594
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2247 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2247 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2247 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2247 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2247 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2247 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2247 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2247 [00:00<?, ?it/s]

In [10]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
# => 훈련되어서 저장된 s-bert 모델을 불러와서 성능 평가 해봄
##############################################################################
import time

# 훈련완료후 테스트해볼 파일 경로(*Kluests_test 파일은 없어서, korsts test 파일 이용)
test_sts = '../../data11/korpora/korsts/tune_test.tsv'
test_batch_size = 32

# 테스트시 cosine 유사도등 측정 결과값 파일 (similarity_evaluation_xxxx.xls) 저장될 경로
test_output_path = "./output"
os.makedirs(test_output_path, exist_ok=True)

test_samples = []
with open(test_sts, 'rt', encoding='utf-8') as fIn:
    lines = fIn.readlines()
    for line in lines:
        s1, s2, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0
        test_samples.append(InputExample(texts=[s1,s2], label=score))

logger.info("\n")
logger.info("======================TEST===================")
logger.info("\n\n")
logger.info(f"model save path > {smodel_path}")
start = time.time()
model = SentenceTransformer(smodel_path)

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=test_batch_size, name='sts-test', show_progress_bar=True)
test_evaluator(model, output_path=test_output_path)
logger.info(f"처리시간 > {time.time() - start:.4f}")

2022-10-13 13:57:56,729 - s-bert - INFO - 

2022-10-13 13:57:56,732 - s-bert - INFO - ======================TEST===================
2022-10-13 13:57:56,732 - s-bert - INFO - 


2022-10-13 13:57:56,733 - s-bert - INFO - model save path > ../../data11/model/sbert/sbert-mdistilbertV3.1.1-disitl-ns-1.5


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

2022-10-13 13:57:59,538 - s-bert - INFO - 처리시간 > 2.8043


In [11]:
# 마지막 model 저장
#test_output_path = "../../data11/model/sbert/test"
#model.save(test_output_path)